In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 登录 URL 和数据获取 URL
login_url = "http://192.168.15.1/cgi-bin/luci"
interface_url = "http://192.168.15.1/cgi-bin/luci/admin/network/network"

# 用户名和密码（根据实际情况更改）
username = "admin"
password = "admin"


# 初始化 Chrome 浏览器
driver = webdriver.Chrome()

# 访问登录页面
driver.get(login_url)

# 填写登录表单
# username_input = driver.find_element(By.ID, 'luci_username')
password_input = driver.find_element(By.ID, 'luci_password')

# username_input.send_keys(username)
password_input.send_keys(password)

# 点击登录按钮
login_button = driver.find_element(By.TAG_NAME, 'button')
login_button.click()


In [24]:
# 登录成功后，访问接口页面
driver.get(interface_url)

# 等待页面 JavaScript 执行，并等待 id="cbi-network-interface" 的 div 出现
# try:
network_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'cbi-network-interface')))
# 查找包含 class="table cbi-section-table" 的表格
table = network_div.find_element(By.CLASS_NAME, 'cbi-section-table')
# 查找所有表格行
rows = table.find_elements(By.TAG_NAME, 'tr')

In [25]:
vwan_ips = {}  # 存储接口名称和对应的IP地址

for row in rows:
    try:
        # 找到 class="ifacebox-head" 的 div 并获取其文本
        ifacebox_head = row.find_element(By.CSS_SELECTOR, 'div.ifacebox-head')
        iface_name = ifacebox_head.text.strip()

        # 筛选以 'vwan' 开头的接口
        if iface_name.startswith('vwan'):
            # 构造描述 div 的 ID，例如 "vwan1-ifc-description"
            description_div_id = f"{iface_name}-ifc-description"

            # 找到描述 div
            description_div = row.find_element(By.ID, description_div_id)

            # 在描述 div 中找到包含 "IPv4:" 的 span
            ipv4_span = description_div.find_element(
                By.XPATH, ".//span[strong[text()='IPv4: ']]"
            )

            ipv4_text = ipv4_span.text  # 例如 "IPv4: 10.19.127.218/22"

            # 提取 IP 地址部分
            ip_address = ipv4_text.split(': ')[1].split('/')[0]

            # 存储结果
            vwan_ips[iface_name] = ip_address

    except Exception as e:
        print(f"处理行时出错: {e}")

# 输出结果
for iface, ip in vwan_ips.items():
    print(f"接口 {iface} 的 IPv4 地址是: {ip}")

接口 vwan1 的 IPv4 地址是: 10.19.127.218
接口 vwan2 的 IPv4 地址是: 10.19.127.24
接口 vwan3 的 IPv4 地址是: 10.19.126.63
接口 vwan4 的 IPv4 地址是: 10.19.127.241
接口 vwan5 的 IPv4 地址是: 10.19.126.108


In [26]:
driver.close()